In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import math

In [18]:
# Read in text data

reddit_data = pd.read_csv("/Users/eugen/Desktop/Work/BrainStation/Suicide Project/Suicide_Detection.csv", index_col = 0).reset_index(drop=True) #with open()
reddit_data.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


## Data Pre-processing and Cleaning

In [19]:
print(f"Dataset has {reddit_data.shape[0]} and {reddit_data.shape[1]} columns")

Dataset has 232074 and 2 columns


In [20]:
reddit_data.info() #no missing values and both columns are in the right format

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232074 entries, 0 to 232073
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    232074 non-null  object
 1   class   232074 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [21]:
reddit_data.text.unique()

array(["Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.",
       "Am I weird I don't get affected by compliments if it's coming from someone I know irl but I feel really good when internet strangers do it",
       'Finally 2020 is almost over... So I can never hear "2020 has been a bad year" ever again. I swear to fucking God it\'

The text column contains reddit posts for the users; however, it is very messy. We need to thoroughly clean each text because our analysis and model performance will depend on it. Let's take a look at a few 